#### Import sympy for calculations. 

Just Google it. 

`conda update sympy` worked for me.

In [1]:
from sympy import *
from sympy.physics.quantum import Ket

#### Define bras, kets, brackets
$
\newcommand{\ket}[1]{\left|{#1}\right\rangle}
\newcommand{\bra}[1]{\left\langle{#1}\right|}
\newcommand{\braket}[2]{\left\langle{#1}\middle|{#2}\right\rangle}
$

# Quantum Computation and Quantum Information

### Exercise 2.79 (Wayne Dam)

Consider a composite of two qubits. Find the Schmidt decompostion of the following states.

1. $\frac{1}{\sqrt{2}}(\ket{00} + \ket{11})$ 

2. $\frac{1}{2} (\ket{00}+\ket{01}+\ket{10}+\ket{11})$

3. $\frac{1}{\sqrt{3}}(\ket{00} + \ket{01} + \ket{10})$

#### Solution
From the proof of Theorem 2.7 for the Schmidt decomposition we write the original pure state of the composite system in the form:

$\ket{\psi} = \sum_{jk} a_{jk} \ket{j}\ket{k}.$

Considering the coefficients $a_{jk}$ as elements of a matrix, $a$, we can take its singular value decomposition (SVD), $a=udv \rightarrow a_{jk} = u_{ji}d_{ii}v_{ik}$.

NOTE: The SVD is usually written with the $v$ as its conjugate transpose: $a = udv^\dagger.$ Numercial and symbolic SVD solvers like sympy, which we'll use below, usually assume the conjugate transpose version. This must be taken into account.

Following the proof and its notation regarding $v$, we can write the new state vectors for the Schmidt decomposition as:
$\ket{i_A} = \sum_j u_{ji} \ket{j}$

$\ket{i_B} = \sum_k v_{ik} \ket{k}.$

Staring at these equations and picturing the $u$ and $v$ matrices, you see that for $\ket{i_A}$ we are just picking coefficients from $i$'th column of $u$ to generate the $\ket{i_A}$. Similarly for $v$ and $\ket{i_B}$ but for the $i$'th row. But for the typical solver returning the daggered form we will also just be running down the returned $i$'th column.

So using Sympy this allows us an easy automated calculation.

In [18]:
# For the first problem it is already in Schmidt form, so this is a good check on the method.

# Put the coefficients of the pure state in matrix form.
A = 1/sqrt(2) * Matrix([[1, 0],[0, 1]])
display(A)

Matrix([
[sqrt(2)/2,         0],
[        0, sqrt(2)/2]])

In [19]:
# Now do the SVD.
U, S, V = A.singular_value_decomposition()

In [20]:
display(U)
display(S)
display(V)

Matrix([
[1, 0],
[0, 1]])

Matrix([
[sqrt(2)/2,         0],
[        0, sqrt(2)/2]])

Matrix([
[1, 0],
[0, 1]])

In [5]:
display(U.evalf())
display(S.evalf())
display(V.evalf())

Matrix([
[1.0,   0],
[  0, 1.0]])

Matrix([
[0.707106781186548,                 0],
[                0, 0.707106781186548]])

Matrix([
[1.0,   0],
[  0, 1.0]])

In [6]:
# Get the Schmidt number.
sn = S.shape[0]
sn

2

In [7]:
# Get the dimension of the Hilbert space.
d = A.shape[0]
d

2

In [8]:
# This is optional. The third part of this question is complicated in closed form.
# No reason here not to just go to numerical values in that case.
# Skip this step if you want to continue in exact values.

# U = U.evalf()
# V = V.evalf()
# S = S.evalf()

In [9]:
# Based on the discusion above, just grab the coeffients down the columns of $u$ 
# to get the new Schmidt basis vectors for system A.
ketsA = [sum([U[j,i]*Ket(j) for j in range(d)]) for i in range(sn)]
ketsA

[|0>, |1>]

In [10]:
# Now do the same for system B using the v matrix. 
# We do the column because Sympy returned the conjugate transposed V.
ketsB = [sum([V[k,i]*Ket(k) for k in range(d)]) for i in range(sn)]
ketsB

[|0>, |1>]

By multiplying corresponding elements of the two lists and summing along with the Schmidt coefficients (diagonal elemens of $s$) we get the expected answer:

$\ket{\psi} = \frac{1}{\sqrt{2}}(\ket{00} + \ket{11})$ 

The Schmidt number is 2 and the Schmidt coefficients are $1/\sqrt{2}$ each.

In [11]:
# Check to see that we get the original expression after substituion and expanding.
# Note that kets exponentiated by 2 just mean the index is duplicated. It's a Sympy thing.
# So \ket{0}**2 means \ket{00}.
sum( [expand(S[i,i] * ketsA[i]*ketsB[i]) for i in range(sn)] )

sqrt(2)*|0>**2/2 + sqrt(2)*|1>**2/2

**We can wrap this up now in a convenient function.**

You can see how this can be easily turned into a function. There's no issue with using higher dimensional Hilbert space as it is. The manual step right now is writing the original pure state into the  𝑎 matrix.

In [12]:
def schmidt_decomposition(A, do_float):
    # A: Matrix holding the coefficients of the pure state to put into Schmidt form.
    # do_float: The third problem introduces complicated analytical terms. 
    #           This just forces the results into floating point for clarity.
    
    # Do the SVD.
    U, S, V = A.singular_value_decomposition()
    
    display('U = ', U)
    display('S = ', S)
    display('V = ', V)
    display('===================================')
    display('U = ', U.evalf())
    display('S = ', S.evalf())
    display('V = ', V.evalf())
    display('===================================')    
    
    # Get the Schmidt number and dimension we are working in.
    sn = S.shape[0]
    print('Schmidt number: ', sn)
    d = A.shape[0]
    print('Dimension d:    ', d)
    display('===================================')

    # This is optional. The third part of this question is complicated in closed form.
    # No reason here not to just go to numerical values in that case.
    # Skip this step if you want to continue in exact values.
    if do_float:
        U = U.evalf()
        V = V.evalf()
        S = S.evalf()
    
    # Based on the discusion in the solution, just grab the coeffients down the columns of $u$ 
    # to get the new Schmidt basis vectors for system A.
    ketsA = [sum([U[j,i]*Ket(j) for j in range(d)]) for i in range(sn)]
    display('ketsA = ', ketsA)
    
    # Now do the same for system B using the v matrix. 
    # We do the column because Sympy returned the conjugate transposed V.
    ketsB = [sum([V[k,i]*Ket(k) for k in range(d)]) for i in range(sn)]
    display('ketsB = ', ketsB)
    display('===================================')
    
    # Check to see that we get the original expression after substituion and expanding.
    # Note that kets exponentiated by 2 just mean the index is duplicated. It's a Sympy thing.
    # So \ket{0}**2 means \ket{00}.
    expansion_check = sum( [expand( S[i,i] * ketsA[i]*ketsB[i] ) for i in range(sn)] )
    display('Expansion check: ', expansion_check)

In [13]:
A = 1/sqrt(2) * Matrix([[1, 0],[0, 1]])
display(A)

schmidt_decomposition(A, do_float=false)

Matrix([
[sqrt(2)/2,         0],
[        0, sqrt(2)/2]])

'U = '

Matrix([
[1, 0],
[0, 1]])

'S = '

Matrix([
[sqrt(2)/2,         0],
[        0, sqrt(2)/2]])

...

'ketsB = '

[|0>, |1>]

'==================================='

'Expansion check: '

sqrt(2)*|0>**2/2 + sqrt(2)*|1>**2/2

In [14]:
A = 1/2 * Matrix([[1, 1],[1, 1]])
display(A)

schmidt_decomposition(A, do_float=false)

Matrix([
[0.5, 0.5],
[0.5, 0.5]])

'U = '

Matrix([
[-0.707106781186548],
[-0.707106781186548]])

'S = '

Matrix([[1.0]])

...

'ketsB = '

[-0.707106781186548*|0> - 0.707106781186548*|1>]

'==================================='

'Expansion check: '

0.5*|0>*|1> + 0.5*|0>**2 + 0.5*|1>*|0> + 0.5*|1>**2

In [15]:
A = 1/sqrt(3) * Matrix([[1, 1],[1, 0]])
display(A)

schmidt_decomposition(A, do_float=true)

Matrix([
[sqrt(3)/3, sqrt(3)/3],
[sqrt(3)/3,         0]])

'U = '

Matrix([
[(sqrt(3)*(1/2 - sqrt(5)/2)/(3*sqrt((-1/2 + sqrt(5)/2)**2 + 1)) + sqrt(3)/(3*sqrt((-1/2 + sqrt(5)/2)**2 + 1)))/sqrt(1/2 - sqrt(5)/6), (sqrt(3)/(3*sqrt(1 + (1/2 + sqrt(5)/2)**2)) + sqrt(3)*(1/2 + sqrt(5)/2)/(3*sqrt(1 + (1/2 + sqrt(5)/2)**2)))/sqrt(sqrt(5)/6 + 1/2)],
[                                                sqrt(3)*(1/2 - sqrt(5)/2)/(3*sqrt(1/2 - sqrt(5)/6)*sqrt((-1/2 + sqrt(5)/2)**2 + 1)),                                                sqrt(3)*(1/2 + sqrt(5)/2)/(3*sqrt(1 + (1/2 + sqrt(5)/2)**2)*sqrt(sqrt(5)/6 + 1/2))]])

'S = '

Matrix([
[sqrt(1/2 - sqrt(5)/6),                     0],
[                    0, sqrt(sqrt(5)/6 + 1/2)]])

...

'ketsB = '

[-0.525731112119134*|0> + 0.85065080835204*|1>,
 0.85065080835204*|0> + 0.525731112119134*|1>]

'==================================='

'Expansion check: '

0.577350269189626*|0>*|1> + 0.577350269189626*|0>**2 + 0.577350269189626*|1>*|0>

**Summary**

For part 2 our $a$ matrix is now: 

In [16]:
A = 1/2 * Matrix([[1, 1],[1, 1]])
display(A)

Matrix([
[0.5, 0.5],
[0.5, 0.5]])

Replacing the $a$ matrix in the code above gives us:

The system has a Schmidt number of 1.
The Schmidt coefficient equals 1 exactly.

$\ket{0_A} = -\frac{1}{\sqrt{2}} (\ket{0}_A + \ket{1}_A)$

$\ket{0_B} = -\frac{1}{\sqrt{2}} (\ket{0}_B + \ket{1}_B)$


Note I've added subcripts of $A$ and $B$ on the rhs to emphasize with system the original basis vectors are coming from. 
Substituting and expanding in the code above again gives back the original pure state in the original basis.

For part 3 our  𝑎 matrix is now:

In [17]:
A = 1/sqrt(3) * Matrix([[1, 1],[1, 0]])
display(A)

Matrix([
[sqrt(3)/3, sqrt(3)/3],
[sqrt(3)/3,         0]])

Replacing the 𝑎 matrix in the code above gives us:

The system has a Schmidt number of 2. 

The Schmidt coefficients are:

$\sqrt{1/2 - \sqrt{5}/6}$

$\sqrt{1/2 + \sqrt{5}/6)}$

The Schmidt basis are:

$\ket{0_A} = 0.525731112119134\ |0> - 0.85065080835204\ |1>$
$\ket{1_A} = 0.85065080835204\ |0> + 0.525731112119134\ |1>$

$\ket{0_B} = -0.525731112119134\ |0> + 0.85065080835204\ |1>$
$\ket{1_B} =  0.85065080835204\ |0> + 0.525731112119134\ |1>$
 
Again, substituting and expanding in the code above again gives back the original pure state in the original basis.

You can see how this can be easily turned into a function. There's no issue with using higher dimensional Hilbert space as it is. The manual step right now is writing the original pure state into the $a$ matrix.